In [1]:
import clingo
import re
from json import dumps
import os

In [2]:
def combine(encoding_path, benchmark_path, destination_path):  
    combined_path = destination_path
    
    encoding_file = open(encoding_path, "r")
    encoding = encoding_file.read()
    encoding_file.close()
    
    benchmark_file = open(benchmark_path, "r")
    benchmark = benchmark_file.read()
    benchmark_file.close()
    
    combined_file = open(combined_path, "w")
    combined = combined_file.write(benchmark + encoding)
    combined_file.close()

In [3]:
def save(solution, destination_path):
    file = open(destination_path, "w")
    file.write(solution)
    file.close()

In [4]:
def save_plans(plans, destination_path):
    file = open(destination_path, "w")
    for i in range(len(plans)):
        file.write(plans[i])
    file.close()

In [5]:
def solve(robot_id, man_dis, destination_path, time, restrictions=[]):
    combined_path = destination_path
    combined_file = open(combined_path, "r")
    combined = combined_file.read()
    combined_file.close()
    rid = "#const rid = {}.".format(robot_id)
    md=man_dis
    if md==0:
        while md==0:
            solution = ""
            horizon = "#const horizon = {}.".format(md)
            asp = horizon + "\n" + rid + "\n" + combined
            if restrictions:
                for j in restrictions:
                    if j:
                        asp = asp + "\n" + j

            ctl = clingo.Control(['--stats'])
            ctl.add("base", [], asp)
            ctl.ground([("base", [])])
            with ctl.solve(yield_=True) as handle:
                for m in handle: solution=("{} ".format(m))
                handle.get()
            time.append(float(dumps(ctl.statistics['summary']['times']['total'], sort_keys=True, indent=4, separators=(',', ': '))))
            solution = solution.replace(' ', '. ')
            md=md+1
            if solution:
                return solution, md, time
                break

    for i in range(md,md*2):
        solution = ""
        horizon = "#const horizon = {}.".format(i)
        asp = horizon + "\n" + rid + "\n" + combined
        if restrictions:
            for j in restrictions:
                if j:
                    asp = asp + "\n" + j
        ctl = clingo.Control(['--stats'])
        ctl.add("base", [], asp)
        ctl.ground([("base", [])])
        with ctl.solve(yield_=True) as handle:
            for m in handle: solution=("{} ".format(m))
            handle.get()
        time.append(float(dumps(ctl.statistics['summary']['times']['total'], sort_keys=True, indent=4, separators=(',', ': '))))
        solution = solution.replace(' ', '. ')
        if solution:
            return solution, i, time
            break

In [6]:
def prep(destination_path, time):
    combined_path = destination_path
    combined_file = open(combined_path, "r")
    combined = combined_file.read()
    combined_file.close()
    
    solution = ""
    md=[]
    r=[]
    asp=combined

    ctl = clingo.Control(['--stats'])
    ctl.add("base", [], asp)
    ctl.ground([("base", [])])
    with ctl.solve(yield_=True) as handle:
        for m in handle: 
            solution=("{} ".format(m))
            md=re.findall('md(.+?) ', ("{} ".format(m)))
            r=re.findall('rbt(.+?) ', ("{} ".format(m)))
        handle.get()
        solution = solution.replace(' ', '. ')
    time.append(float(dumps(ctl.statistics['summary']['times']['total'], sort_keys=True, indent=4, separators=(',', ': '))))
    return solution, r, md, time

In [7]:
def preplan(robots, man_dis, destination, time, restrictions=[]):
    plans=[]
    plan_=""
    paths = []
    actions = []
    
    if not paths:
        for i in robots:
            paths.append([])
    
    for i in robots:
        temp = solve(i, man_dis, destination, time, restrictions)
        paths[i-1].append(temp[0])
        actions.append(temp[1])
        time = temp[2]
        if paths[i-1][0]:
            plan_ = plan_ + paths[i-1][0]
    return paths, plan_, actions, time

In [8]:
def collisions(robot_id, encoding_path, plan, time):
    
    encoding_file = open(encoding_path, "r")
    encoding = encoding_file.read()
    encoding_file.close()
    
    rid = "#const rid = {}.".format(robot_id)
    asp = rid + "\n" + plan + "\n" + encoding
    solution_ = ""
    li = []
    
    ctl = clingo.Control(['--stats'])
    ctl.add("base", [], asp)
    ctl.ground([("base", [])])
    with ctl.solve(yield_=True) as handle:
        for m in handle: 
            solution_=("{} ".format(m))
            li = re.findall('s\((.+?),',("{} ".format(m)))
        handle.get()
    time.append(float(dumps(ctl.statistics['summary']['times']['total'], sort_keys=True, indent=4, separators=(',', ': '))))
    li=list(set(map(int,li)))
    if solution_== " ":
        return
    else:
        solution_ = solution_.replace(' ', '. ')
        
        return solution_, li, time

In [9]:
def plan(robots, destination, collision, paths, plan_, actions, time, restrictions=[]):
    max_value = max(actions)
    max_index = actions.index(max_value)
    cur_rest = []
    temp_ = collisions(actions[max_index], collision, plan_, time)
    if temp_:
        cur_rest.append(temp_[0])
        time=temp_[2]
    if cur_rest:
        restrictions.append(cur_rest[-1])
        restricted_robots = temp_[1]
        for i in restricted_robots:
            temp = solve(i, max_value, destination, time, restrictions)
            if temp:
                paths[i-1] = [temp[0]]
                actions[i-1] = temp[1]
                time = temp[2]
        plan_ = ""
        for i in robots:
            if paths[i-1][0]:
                plan_ = plan_ + paths[i-1][0]
        plan_, time = plan(robots, destination, collision, paths, plan_, actions, time, restrictions)
    return plan_, time

In [10]:
enc1 = "./asprilo-encodings/recursive/path.lp"
enc2 = "./asprilo-encodings/recursive/prep.lp"
ben = "./instances/adrian/benchmark2/CNC/cnc_instance.lp"
dest1 = "./solution/recursive/combined_instance.lp"
dest2 = "./solution/recursive/combined_prep.lp"
dest3 = "./solution/recursive/prep.lp"
col = "./asprilo-encodings/collision/collision.lp"
dest4 = "./solution/recursive/plan.lp"
time=[]
rest=[]

In [11]:
combine(enc2, ben, dest2)

prep_, robots, man_dis, time = prep(dest2, time)

save(prep_,dest3)
combine(enc1,dest3,dest1)


for i in range(len(robots)):
    robots[i] = int(robots[i].strip('()'))
md=[]
for i in range(len(man_dis)):
    x,y = map(int, man_dis[i].strip('()').split(','))
    md.append(y)
max_md = max(md)
min_md = min(md)
paths, plan_, actions, time = preplan(robots, max_md, dest1, time, rest)

In [12]:
plans, time = plan(robots, dest1, col, paths, plan_, actions, time, rest)

In [13]:
save_plans(plans, dest4)

In [14]:
print(plans)

position(1,(14,5),0). position(1,(13,5),1). position(1,(12,5),2). position(1,(12,5),3). position(1,(12,5),4). position(1,(12,6),5). position(1,(12,6),6). position(1,(12,6),7). position(1,(12,6),8). position(1,(12,6),9). position(1,(12,6),10). position(1,(12,6),11). position(1,(12,6),12). position(1,(12,6),13). position(1,(12,6),14). position(1,(12,6),15). position(1,(12,6),16). position(1,(12,6),17). position(1,(12,6),18). position(1,(12,6),19). position(1,(12,6),20). position(1,(12,6),21). position(1,(12,6),22). occurs(object(robot,1),action(move,(-1,0)),1). occurs(object(robot,1),action(move,(-1,0)),2). occurs(object(robot,1),action(move,(0,1)),5). position(2,(7,13),0). position(2,(8,13),1). position(2,(9,13),2). position(2,(10,13),3). position(2,(11,13),4). position(2,(12,13),5). position(2,(12,12),6). position(2,(12,11),7). position(2,(12,10),8). position(2,(12,9),9). position(2,(12,9),10). position(2,(12,9),11). position(2,(12,9),12). position(2,(12,9),13). position(2,(12,9),14). 

In [15]:
print(sum(time))

3.916867971420288


In [16]:
try:                                                            # delete temporary files
    os.remove(dest1)
    os.remove(dest2)
    os.remove(dest3)
except OSError as e:
    print("Error: %s : %s" % (dest1, e.strerror))
    print("Error: %s : %s" % (dest2, e.strerror))
    print("Error: %s : %s" % (dest3, e.strerror))